In [7]:
import import_ipynb
import pandas as pd
import requests
from multiprocessing.dummy import Pool
import re
from SPARQLWrapper import SPARQLWrapper, JSON
from collections import defaultdict
from knowledge_extract import *
from os import listdir
from os.path import isfile, join
import traceback
from datetime import date
import datetime

In [ ]:
#%lsmagic

In [ ]:
%%time
people=pd.read_csv('./neurotree/people_cs.tsv', sep='\t', encoding='utf-8',  escapechar='\\')
connect=pd.read_csv('./neurotree/connect_cs.tsv', sep='\t', encoding='latin-1',  escapechar='\\')
people['name']=people['firstname'].fillna('')+" "+people['middlename'].fillna('')+" "+people['lastname'].fillna('')
#people['name']=people['name'].str.replace('.','',regex=False)
people['name']=people['name'].str.replace(' +',' ',regex=True)

In [ ]:
people.shape

In [ ]:
people[people['name'].str.contains('Anup Kumar')]

In [ ]:
def get_id(args):
    ent=get_entity_id(args[1],"https://www.wikidata.org/w/api.php")
    return (args[0], args[1], ent)
result=[]

In [10]:
'./tmp'+date.today().strftime("%d-%m-%Y_%H:%M:%S")+'/'

'./tmp14-09-2019_00:00:00/'

In [9]:
datetime.datetime.now().strftime("%d-%m-%Y %H:%M:%S")

'11-09-2019 09:54:20'

In [ ]:
%%time
names=people[['pid','name']]
names = [tuple(x) for x in names.values]
p=Pool(3)
for a in p.imap_unordered(get_id, names):
        result.append(a)
mod_result=[t for t in result if t[2] is not None]
with open('pid2ent2.json', 'w') as fp:
    json.dump(mod_result, fp, sort_keys=True, indent=4)

In [ ]:
len(mod_result)

In [ ]:
b=[]
for a in mod_result:
    b.append(a[2])

In [ ]:
len(set(b))

In [2]:
def create_inst(entity):
    global directory
    props=['P27','P106','P101','P184','P185','P1026','P802','P1066','P800','P856','P2456','P2381','P864',
       'P108','P69','P512','P166','P463','P39','P496']
    try:
        inst=Knowledge_extract(entity[2], props)
        if not inst.flag:
            inst.write_json(directory=directory)
    except Exception as e:
        print(entity[2])
    return

In [3]:
def load_json(filename, directory='./tmp/'):
    path=os.path.join(directory, filename + "." + 'json')
    with open(path) as f:
        data = json.load(f)
    return data

In [ ]:
def read_filesInDir(directory):
    jsonfiles = [f for f in listdir(directory) if isfile(join(directory, f))]
    entity=[f.split('.')[0] for f in jsonfiles]
    return entity

In [4]:
%%time
directory='./tmp'+date.today().strftime("%d_%m_%Y")+'/'
Knowledge_extract.get_entities(directory=directory)
res=load_json('pid2ent2','./')
p=Pool(3)
try:
    p.map(create_inst,res)
except Exception as e:
    traceback.print_exc()

Q15637844
CPU times: user 12min 56s, sys: 1min 5s, total: 14min 2s
Wall time: 2h 44min 15s


In [ ]:
for a in mod_result:
    if a[1]=="Anup Kumar":
        print(a)

In [ ]:
result

In [ ]:
a=[]
for r in mod_result:
    a.append(r[2])
b=set(a)
dup=[]
for e in b:
    count=a.count(e)
    if count > 1:
        dup.append(e)

In [ ]:
for d in dup:
    for r in mod_result:
        if r[2]==d:
            print(r)

In [ ]:
mod_result